In [1]:
import re
import time
import pandas as pd
from tqdm import tqdm
from random import random

import os
import requests
from bs4 import BeautifulSoup
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

In [2]:
films_per_category = 350
films_per_page = 50
categories = [
    "action",
    "adventure",
    "animation",
    "biography",
    "comedy",
    "crime",
    "documentary",
    "drama",
    "family",
    "fantasy",
    "film_noir",
    "history",
    "horror",
    "music",
    "musical",
    "mystery",
    "romance",
    "sci_fi",
    "short",
    "sport",
    "superhero",
    "thriller",
    "war",
    "western"
    ]

title_root = 'https://www.imdb.com/title/'
genre_root = 'https://www.imdb.com/search/title/?genres='
superhero_root = 'https://www.imdb.com/search/keyword/?keywords=superhero&title_type=movie&page='

In [3]:
path_to_df = "./data/raw_data.csv"
path_to_images = "./data/images/"

In [5]:
def get_html(url):
    time.sleep(random() * 3)
    response = session.get(url, headers= {"User-Agent":"Mozilla/5.0 (X11; U; Linux i686) Gecko/20071127 Firefox/2.0.0.11", 'Accept-Language': 'en'})
    return BeautifulSoup(response.content, 'html.parser')

def get_links_to_films(html):
    divs = html.findAll("div", {"class": "lister-item-image"})
    a_sub_tags = [div.findAll("a", href=True) for div in divs]
    a_tags = [a_tag for nested_list in a_sub_tags for a_tag in nested_list]
    hrefs = [tag["href"] for tag in a_tags]
    filtered_hrefs = [href for href in hrefs if re.compile("^\/title\/tt.*").match(href)]
    title_ids = [re.compile("tt\d*").search(id).group(0) for id in filtered_hrefs]
    return title_ids

In [6]:
def parse_genre_ids(genre):
    unique_ids = []
    for i in tqdm(range(1, films_per_category, films_per_page)):
        # Superhero is more tag than genre
        if genre != "superhero":
            url = genre_root + genre.lower() + "&start=" + str(i)
        else:
            url = superhero_root + str(i // films_per_page + 1)
        html = get_html(url)
        unique_ids.extend(get_links_to_films(html))

    return unique_ids

def parse_film_page(url):
    html = get_html(url)
    while not str(html.find("h1").contents[0]):
        html = get_html(url)
    title = str(html.find("h1", {"class": "sc-b73cd867-0"}).contents[0])
    poster_url = str(html.find("img", {"class": "ipc-image"})["src"]) if html.find("img",
                                                                                    {"class": "ipc-image"}) else ""
    synopsis = html.find("span", {"role": "presentation", "class": "sc-16ede01-2"})
    synopsis = str(synopsis.contents[0].string) if synopsis and len(
        synopsis.contents) else ""
    genre = " ".join([item.contents[0] for item in html.findAll("span", {"class": "ipc-chip__text"})])
    return title, poster_url, synopsis, genre

def parse_genre(genre):
    title_list = []
    poster_url_list = []
    synopsis_list = []
    genre_list = []
    poster_location = []

    unique_ids = parse_genre_ids(genre)
    print(f"Collected number films id: {len(unique_ids)}")
    for id in tqdm(unique_ids):
        url = title_root + id
        title, poster_url, description, labels = parse_film_page(url)

        title_list.append(title)
        poster_url_list.append(poster_url)
        synopsis_list.append(description)
        genre_list.append(labels)

        if poster_url:
            poster_response = session.get(poster_url)
            if poster_response.status_code == 200:
                path_to_save = path_to_images + genre + "/" + re.sub("\\W", "_", title.casefold()) + ".jpg"
                os.makedirs(os.path.dirname(path_to_save), exist_ok=True)
                with open(path_to_save, 'w+b') as f:
                    f.write(poster_response.content)
                    poster_location.append(path_to_save)
            else:
                print(f"{poster_response}. Urs: {url}")
        else:
            poster_location.append(' ')

    return title_list, poster_url_list, synopsis_list, genre_list, poster_location

In [ ]:
session = requests.Session()
retry = Retry(connect=5, backoff_factor=1)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

In [8]:
title_list = []
genre_list = []
synopsis_list = []
poster_url_list = []
poster_location_list = []
with_sub_genre_list = []

for genre in tqdm(categories):
    print(f"Process genre: {genre}")
    genre_title_list, genre_poster_url_list, genre_synopsis_list, genre_cats_list, genre_poster_location = parse_genre(genre)
    title_list.extend(genre_title_list)
    poster_url_list.extend(genre_poster_url_list)
    synopsis_list.extend(genre_synopsis_list)
    genre_list.extend([genre] * len(genre_cats_list))
    poster_location_list.extend(genre_poster_location)
    with_sub_genre_list.extend(genre_cats_list)


df = pd.DataFrame({
                "title": title_list,
                "poster_url": poster_url_list,
                "synopsis": synopsis_list,
                "genre": genre_list,
                "poster_location": poster_location_list
                })

df.to_csv(path_to_df, index=False)


  0%|          | 0/16 [00:00<?, ?it/s]

Process genre: family


100%|██████████| 7/7 [00:19<00:00,  2.83s/it]


Collected number films id: 350


  6%|▋         | 1/16 [19:38<4:54:37, 1178.52s/it]

...

 94%|█████████▍| 15/16 [5:01:13<19:44, 1184.78s/it]

Process genre: western


100%|██████████| 7/7 [00:22<00:00,  3.18s/it]


Collected number films id: 350


100%|██████████| 16/16 [5:21:17<00:00, 1204.85s/it]
